In [1]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re

In [2]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [3]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
#rochelle.head()


In [4]:
rochelle.columns

Index(['Scientific Name', 'Taxon', 'List', 'GLANSIS Authors',
       'Assessment Documented', 'Review by:', 'Shared with USGS',
       'NOAA EndNote Library Status', 'Assigned to', 'Action Needed', 'Notes',
       'Priority', 'References Entered and Checked (Date)',
       'Map Updated (Date + partial or complete)', 'Impact Data Entered',
       'Demographics Table Populated', 'Demographics Table Sources',
       'OIA/RA Complete', 'Draft Profile Update (Ready for Review)',
       'Review Status', 'Internal NOAA Reviewer(s)', 'USGS Reviewer?',
       'Past External Reviewers', 'New External Reviewers',
       'Review complete and Cleared to post',
       'Date Profile Last Updated  (Posted online)',
       'Last Review Date (GLANSIS Records)', 'OIA/RA Posted to Clearinghouse',
       'Twitter'],
      dtype='object')

In [5]:
#rochelle.reset_index(inplace = True,drop = True) 
#name = rochelle.loc[0]["Scientific Name"] #for some reason couldn't access rochelle[0]...
#print(name)


In [6]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [7]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
# url = "https://quod.lib.umich.edu/h/herb00ic/x-1000138/MICH-V-1000138A?lasttype=boolean;lastview=reslist;op2=And;q2=Phragmites+australis+australis;resnum=1;rgn2=ic_all;select2=all;size=50;start=1;subview=detail;view=entry"
#html = urllib.request.urlopen(url, context=ctx).read()
#soup = BeautifulSoup(html, 'html.parser')
#soup

In [8]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
#resp = requests.get(url)
#newsoup = BeautifulSoup(resp.content, 'html.parser')
#iteminfo = soup.find('dl')
#print(iteminfo)

In [9]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [18]:
#setting up driver for Selenium
driver = webdriver.Firefox()

#send driver to the herbarium website and submit a search query and scrape the amount of results
driver.get(herbarium_url)
elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
elem.send_keys('Cabomba caroliniana') #Sends in a name from Rochelle's list
searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
searchbutton.click()

resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text
#next line returns a list of two numbers like ['23','23']
resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results

if resultcountnumberlist[1]<= 50:
    for result in range(int(resultcountnumberlist[0])): #properly traverses the amount of results on the page
        #print(result)
        resultitem = driver.find_element(by = By.CSS_SELECTOR, value='#maincontent > div.\[.flex.flex-flow-rw.\]\[.flex-gap-1.\] > div.main-panel > section:nth-child(4) > a')
        resultitem.click()

elif resultcountnumberlist[1]>50:
    print('There is More than One Page of Results')
    

#individually select each search result and scrape each matching entry

NoSuchElementException: Message: Unable to locate element: #maincontent > div.\[.flex.flex-flow-rw.\]\[.flex-gap-1.\] > div.main-panel > section:nth-child(4) > a
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16
